# TP 05: Trajectory Planning and Joint Control

**Course:** Robotics - 3rd Year GEII  
**Duration:** 4h + 4h (2 sessions)  
**Prerequisites:** TPs 01-04 (Transformations, FK, IK, Jacobian)

---

## Learning Objectives

After completing this lab, you will be able to:

1. **Understand trajectory generation** in joint space and Cartesian space
2. **Design polynomial trajectories** (cubic, quintic) with boundary conditions
3. **Implement trapezoidal velocity profiles** for smooth motion
4. **Apply basic control strategies** (PID control) for joint tracking
5. **Verify and analyze** robot motion using robotics-toolbox-python

---

## Session 1 Content (4 hours)

- **Exercise 1:** Polynomial Trajectory Planning (Joint Space)
- **Exercise 2:** Trapezoidal Velocity Profile
- **Exercise 3:** Multi-Segment Trajectories

## Session 2 Content (4 hours)

- **Exercise 4:** Cartesian Space Trajectories
- **Exercise 5:** PID Joint Control
- **Exercise 6:** Complete Application - Pick and Place

---

## Introduction

In previous TPs, you learned how to compute robot poses (FK), find joint configurations for desired poses (IK), and analyze velocities (Jacobian). However, **moving a robot from one configuration to another** requires planning a smooth path and controlling the joints to follow that path.

This TP covers two fundamental aspects:

### 1. **Trajectory Planning**
Planning smooth paths that:
- Start and end at specified positions
- Have continuous velocities and accelerations
- Respect velocity/acceleration limits
- Avoid jerky motions that can damage the robot

### 2. **Joint Control**
Tracking the planned trajectory using feedback control:
- Measure actual joint positions
- Compare with desired positions
- Apply control torques to minimize errors

---

## Theory Review

### Polynomial Trajectories

A polynomial trajectory for joint $i$ has the form:

$$q_i(t) = a_0 + a_1 t + a_2 t^2 + a_3 t^3 + ... + a_n t^n$$

**Cubic polynomial** (n=3): 4 coefficients → can satisfy 4 boundary conditions:
- Initial position: $q(0) = q_0$
- Final position: $q(T) = q_f$
- Initial velocity: $\dot{q}(0) = \dot{q}_0$
- Final velocity: $\dot{q}(T) = \dot{q}_f$

**Quintic polynomial** (n=5): 6 coefficients → can also control accelerations:
- Initial/final positions, velocities, and accelerations

### Trapezoidal Velocity Profile

Instead of polynomial, use constant acceleration phases:
1. **Acceleration phase:** Linear velocity increase
2. **Constant velocity phase:** Cruise at maximum speed
3. **Deceleration phase:** Linear velocity decrease

### PID Control

For each joint:

$$\tau = K_p e + K_i \int e \, dt + K_d \dot{e}$$

where:
- $e = q_{desired} - q_{actual}$ is the tracking error
- $K_p, K_i, K_d$ are the proportional, integral, and derivative gains
- $\tau$ is the control torque

---

In [ ]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from roboticstoolbox import DHRobot, RevoluteDH, PrismaticDH
from roboticstoolbox import jtraj, ctraj, mtraj
from spatialmath import SE3
import roboticstoolbox as rtb
from scipy.interpolate import CubicSpline

# Configuration for better plots
plt.rcParams['figure.figsize'] = (12, 4)
plt.rcParams['font.size'] = 10

print("Libraries imported successfully!")
print(f"Robotics Toolbox version: {rtb.__version__}")

---

# SESSION 1: Trajectory Planning Fundamentals

---

## Exercise 1: Cubic Polynomial Trajectory (Joint Space)

### Objectives
- Derive cubic polynomial coefficients from boundary conditions
- Implement cubic trajectory generation
- Verify continuity of position and velocity

### Problem Statement

A robot joint must move from $q_0 = 0°$ to $q_f = 90°$ in $T = 2$ seconds.

**Part A: Analytical Solution (BY HAND)**

Given the cubic polynomial:

$$q(t) = a_0 + a_1 t + a_2 t^2 + a_3 t^3$$

And boundary conditions:
- $q(0) = 0°$, $\dot{q}(0) = 0$
- $q(2) = 90°$, $\dot{q}(2) = 0$

1. Write the velocity equation: $\dot{q}(t) = ?$
2. Apply the 4 boundary conditions to obtain 4 equations
3. Solve for coefficients $a_0, a_1, a_2, a_3$
4. Write the final trajectory equation $q(t)$

📝 **Write your solution here (or on paper to submit):**

```
Velocity: ̇q(t) = 

Boundary conditions:
1) q(0) = 0 → 
2) q̇(0) = 0 → 
3) q(2) = 90 → 
4) q̇(2) = 0 → 

Solution:
a0 = 
a1 = 
a2 = 
a3 = 

Final equation:
q(t) = 
```

---

**Part B: Python Implementation**

Implement a function to compute cubic polynomial coefficients.

In [ ]:
def cubic_polynomial_coefficients(q0, qf, q0_dot, qf_dot, T):
    """
    Compute coefficients for cubic polynomial trajectory.
    
    q(t) = a0 + a1*t + a2*t^2 + a3*t^3
    
    Parameters:
    -----------
    q0 : float - Initial position
    qf : float - Final position
    q0_dot : float - Initial velocity
    qf_dot : float - Final velocity
    T : float - Total time duration
    
    Returns:
    --------
    a : numpy array - Coefficients [a0, a1, a2, a3]
    """
    
    # TODO: Implement the coefficient calculation
    # HINT: Set up matrix equation from boundary conditions
    # The boundary conditions give you a system: A * a = b
    
    # Matrix A (from boundary conditions evaluated at t=0 and t=T)
    A = np.array([
        [1, 0, 0, 0],           # q(0) = q0
        [0, 1, 0, 0],           # q_dot(0) = q0_dot
        [1, T, T**2, T**3],     # q(T) = qf
        [0, 1, 2*T, 3*T**2]     # q_dot(T) = qf_dot
    ])
    
    # Vector b (boundary values)
    b = np.array([q0, q0_dot, qf, qf_dot])
    
    # Solve for coefficients
    a = np.linalg.solve(A, b)
    
    return a


def evaluate_cubic_trajectory(a, t):
    """
    Evaluate cubic polynomial and its derivatives.
    
    Parameters:
    -----------
    a : numpy array - Coefficients [a0, a1, a2, a3]
    t : float or array - Time value(s)
    
    Returns:
    --------
    q : position
    q_dot : velocity
    q_ddot : acceleration
    """
    
    # TODO: Implement evaluation of q, q_dot, q_ddot
    q = a[0] + a[1]*t + a[2]*t**2 + a[3]*t**3
    q_dot = a[1] + 2*a[2]*t + 3*a[3]*t**2
    q_ddot = 2*a[2] + 6*a[3]*t
    
    return q, q_dot, q_ddot


# Test your implementation
q0 = np.deg2rad(0)   # Convert to radians
qf = np.deg2rad(90)
q0_dot = 0
qf_dot = 0
T = 2.0

# Compute coefficients
coeffs = cubic_polynomial_coefficients(q0, qf, q0_dot, qf_dot, T)
print("Cubic polynomial coefficients:")
print(f"a0 = {coeffs[0]:.6f}")
print(f"a1 = {coeffs[1]:.6f}")
print(f"a2 = {coeffs[2]:.6f}")
print(f"a3 = {coeffs[3]:.6f}")

# Verify boundary conditions
q_0, qdot_0, _ = evaluate_cubic_trajectory(coeffs, 0)
q_T, qdot_T, _ = evaluate_cubic_trajectory(coeffs, T)

print("\nVerification:")
print(f"q(0) = {np.rad2deg(q_0):.2f}° (should be 0°)")
print(f"q(T) = {np.rad2deg(q_T):.2f}° (should be 90°)")
print(f"q_dot(0) = {qdot_0:.4f} rad/s (should be 0)")
print(f"q_dot(T) = {qdot_T:.4f} rad/s (should be 0)")

**Part C: Visualization**

Plot position, velocity, and acceleration profiles.

In [ ]:
# TODO: Create time vector and evaluate trajectory
t_vec = np.linspace(0, T, 100)
q_vec, qdot_vec, qddot_vec = evaluate_cubic_trajectory(coeffs, t_vec)

# Convert to degrees for plotting
q_deg = np.rad2deg(q_vec)
qdot_deg = np.rad2deg(qdot_vec)
qddot_deg = np.rad2deg(qddot_vec)

# Plot
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

axes[0].plot(t_vec, q_deg, 'b-', linewidth=2)
axes[0].set_xlabel('Time (s)')
axes[0].set_ylabel('Position (degrees)')
axes[0].set_title('Joint Position')
axes[0].grid(True)

axes[1].plot(t_vec, qdot_deg, 'r-', linewidth=2)
axes[1].set_xlabel('Time (s)')
axes[1].set_ylabel('Velocity (deg/s)')
axes[1].set_title('Joint Velocity')
axes[1].grid(True)

axes[2].plot(t_vec, qddot_deg, 'g-', linewidth=2)
axes[2].set_xlabel('Time (s)')
axes[2].set_ylabel('Acceleration (deg/s²)')
axes[2].set_title('Joint Acceleration')
axes[2].grid(True)

plt.tight_layout()
plt.show()

# Find maximum values
print(f"\nTrajectory characteristics:")
print(f"Max velocity: {np.max(np.abs(qdot_deg)):.2f} deg/s")
print(f"Max acceleration: {np.max(np.abs(qddot_deg)):.2f} deg/s²")

**Part D: Quintic Polynomial (BONUS)**

Extend to quintic polynomial with zero initial and final accelerations.

In [ ]:
def quintic_polynomial_coefficients(q0, qf, q0_dot, qf_dot, q0_ddot, qf_ddot, T):
    """
    Compute coefficients for quintic polynomial trajectory.
    q(t) = a0 + a1*t + a2*t^2 + a3*t^3 + a4*t^4 + a5*t^5
    """
    
    # TODO: Implement quintic polynomial (6 boundary conditions)
    A = np.array([
        [1, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0],
        [0, 0, 2, 0, 0, 0],
        [1, T, T**2, T**3, T**4, T**5],
        [0, 1, 2*T, 3*T**2, 4*T**3, 5*T**4],
        [0, 0, 2, 6*T, 12*T**2, 20*T**3]
    ])
    
    b = np.array([q0, q0_dot, q0_ddot, qf, qf_dot, qf_ddot])
    
    a = np.linalg.solve(A, b)
    return a

# Test quintic
coeffs_quintic = quintic_polynomial_coefficients(q0, qf, 0, 0, 0, 0, T)
print("Quintic coefficients:", coeffs_quintic)

### Questions for Exercise 1

✍️ **Answer these questions:**

1. Why do we require zero velocities at start and end?
2. What happens if we use linear interpolation (straight line) instead of cubic?
3. Compare cubic vs quintic: which has smoother acceleration profile?
4. What is the physical meaning of the acceleration profile?

---

## Exercise 2: Trapezoidal Velocity Profile

### Objectives
- Understand bang-bang and trapezoidal profiles
- Design a trajectory with velocity/acceleration constraints
- Compare with polynomial trajectories

### Problem Statement

Design a trapezoidal velocity profile for the same motion: $0° \to 90°$ in 2 seconds.

Constraints:
- Maximum velocity: $v_{max} = 100$ deg/s
- Maximum acceleration: $a_{max} = 150$ deg/s²

**Part A: Analytical Solution (BY HAND)**

The trajectory has 3 phases:

1. **Acceleration phase** ($0 \leq t \leq t_1$): 
   - $a(t) = a_{max}$
   - $v(t) = a_{max} \cdot t$
   - $q(t) = \frac{1}{2} a_{max} t^2$

2. **Constant velocity phase** ($t_1 \leq t \leq t_2$):
   - $a(t) = 0$
   - $v(t) = v_{max}$
   - $q(t) = q(t_1) + v_{max}(t - t_1)$

3. **Deceleration phase** ($t_2 \leq t \leq T$):
   - $a(t) = -a_{max}$
   - $v(t) = v_{max} - a_{max}(t - t_2)$
   - $q(t) = q(t_2) + v_{max}(t - t_2) - \frac{1}{2}a_{max}(t - t_2)^2$

📝 **Calculate by hand:**

Given: $q_0 = 0°$, $q_f = 90°$, $T = 2s$, $a_{max} = 150$ deg/s², $v_{max} = 100$ deg/s

1. Time to reach $v_{max}$: $t_1 = v_{max}/a_{max} = ?$
2. By symmetry, deceleration starts at: $t_2 = T - t_1 = ?$
3. Distance during acceleration: $d_1 = ?$
4. Distance during deceleration: $d_3 = ?$
5. Distance during constant velocity: $d_2 = q_f - d_1 - d_3 = ?$
6. Verify: Is $v_{max}$ achievable? Check if $d_2 \geq 0$

```
YOUR CALCULATIONS:

t1 = 
t2 = 
d1 = 
d2 = 
d3 = 
```

---

**Part B: Python Implementation**

In [ ]:
def trapezoidal_trajectory(q0, qf, T, v_max, a_max):
    """
    Generate trapezoidal velocity profile.
    
    Returns:
    --------
    t_blend : time to reach v_max (acceleration duration)
    v_cruise : actual cruise velocity (may be < v_max)
    """
    
    # Total distance
    distance = abs(qf - q0)
    direction = np.sign(qf - q0)
    
    # Check if we can reach v_max
    # If we accelerate to v_max and decelerate, total distance is:
    # d = 2 * (v_max^2 / (2*a_max)) = v_max^2 / a_max
    # Time spent accelerating and decelerating: t_acc + t_dec = 2 * v_max/a_max
    
    t_acc = v_max / a_max  # Time to reach v_max
    
    # Distance covered during accel + decel
    d_acc_dec = v_max * t_acc  # This is v_max^2 / a_max
    
    if d_acc_dec > distance:
        # Can't reach v_max, triangular profile
        print("Triangular profile (cannot reach v_max)")
        v_cruise = np.sqrt(distance * a_max)
        t_blend = v_cruise / a_max
        t1 = t_blend
        t2 = T - t_blend
    else:
        # Trapezoidal profile
        print("Trapezoidal profile")
        v_cruise = v_max
        t_blend = v_cruise / a_max
        
        # Distance during constant velocity
        d_cruise = distance - d_acc_dec
        t_cruise = d_cruise / v_cruise
        
        t1 = t_blend
        t2 = t1 + t_cruise
    
    return t1, t2, v_cruise * direction, a_max * direction


def evaluate_trapezoidal(t, q0, qf, T, t1, t2, v_cruise, a_max):
    """
    Evaluate trapezoidal trajectory at time t.
    """
    
    if t <= 0:
        return q0, 0, 0
    elif t >= T:
        return qf, 0, 0
    elif t < t1:
        # Acceleration phase
        q = q0 + 0.5 * a_max * t**2
        v = a_max * t
        a = a_max
    elif t < t2:
        # Constant velocity phase
        q = q0 + 0.5 * a_max * t1**2 + v_cruise * (t - t1)
        v = v_cruise
        a = 0
    else:
        # Deceleration phase
        t_dec = t - t2
        q_at_t2 = q0 + 0.5 * a_max * t1**2 + v_cruise * (t2 - t1)
        q = q_at_t2 + v_cruise * t_dec - 0.5 * a_max * t_dec**2
        v = v_cruise - a_max * t_dec
        a = -a_max
    
    return q, v, a


# Test implementation
q0_trap = 0  # degrees
qf_trap = 90
T_trap = 2.0
v_max = 100  # deg/s
a_max = 150  # deg/s²

t1, t2, v_cruise, a_cruise = trapezoidal_trajectory(q0_trap, qf_trap, T_trap, v_max, a_max)

print(f"\nTrapezoidal profile parameters:")
print(f"Acceleration ends at t1 = {t1:.3f} s")
print(f"Deceleration starts at t2 = {t2:.3f} s")
print(f"Cruise velocity = {v_cruise:.2f} deg/s")
print(f"Acceleration = {a_cruise:.2f} deg/s²")

**Part C: Visualization and Comparison**

In [ ]:
# Generate trajectory points
t_vec = np.linspace(0, T_trap, 200)
q_trap = np.zeros_like(t_vec)
v_trap = np.zeros_like(t_vec)
a_trap = np.zeros_like(t_vec)

for i, t in enumerate(t_vec):
    q_trap[i], v_trap[i], a_trap[i] = evaluate_trapezoidal(
        t, q0_trap, qf_trap, T_trap, t1, t2, v_cruise, a_cruise
    )

# Also compute cubic for comparison
coeffs_comp = cubic_polynomial_coefficients(
    np.deg2rad(q0_trap), np.deg2rad(qf_trap), 0, 0, T_trap
)
q_cubic, v_cubic, a_cubic = evaluate_cubic_trajectory(coeffs_comp, t_vec)
q_cubic = np.rad2deg(q_cubic)
v_cubic = np.rad2deg(v_cubic)
a_cubic = np.rad2deg(a_cubic)

# Plot comparison
fig, axes = plt.subplots(3, 1, figsize=(12, 10))

# Position
axes[0].plot(t_vec, q_trap, 'b-', linewidth=2, label='Trapezoidal')
axes[0].plot(t_vec, q_cubic, 'r--', linewidth=2, label='Cubic')
axes[0].axvline(t1, color='gray', linestyle=':', alpha=0.5, label=f't1={t1:.2f}s')
axes[0].axvline(t2, color='gray', linestyle=':', alpha=0.5, label=f't2={t2:.2f}s')
axes[0].set_ylabel('Position (deg)')
axes[0].set_title('Comparison: Trapezoidal vs Cubic Polynomial')
axes[0].legend()
axes[0].grid(True)

# Velocity
axes[1].plot(t_vec, v_trap, 'b-', linewidth=2, label='Trapezoidal')
axes[1].plot(t_vec, v_cubic, 'r--', linewidth=2, label='Cubic')
axes[1].axhline(v_max, color='orange', linestyle='--', label=f'v_max={v_max} deg/s')
axes[1].axvline(t1, color='gray', linestyle=':', alpha=0.5)
axes[1].axvline(t2, color='gray', linestyle=':', alpha=0.5)
axes[1].set_ylabel('Velocity (deg/s)')
axes[1].legend()
axes[1].grid(True)

# Acceleration
axes[2].plot(t_vec, a_trap, 'b-', linewidth=2, label='Trapezoidal')
axes[2].plot(t_vec, a_cubic, 'r--', linewidth=2, label='Cubic')
axes[2].axhline(a_max, color='orange', linestyle='--', label=f'a_max={a_max} deg/s²')
axes[2].axhline(-a_max, color='orange', linestyle='--')
axes[2].axvline(t1, color='gray', linestyle=':', alpha=0.5)
axes[2].axvline(t2, color='gray', linestyle=':', alpha=0.5)
axes[2].set_xlabel('Time (s)')
axes[2].set_ylabel('Acceleration (deg/s²)')
axes[2].legend()
axes[2].grid(True)

plt.tight_layout()
plt.show()

print("\nComparison:")
print(f"Trapezoidal - Max velocity: {np.max(np.abs(v_trap)):.2f} deg/s")
print(f"Cubic - Max velocity: {np.max(np.abs(v_cubic)):.2f} deg/s")
print(f"Trapezoidal - Max acceleration: {np.max(np.abs(a_trap)):.2f} deg/s²")
print(f"Cubic - Max acceleration: {np.max(np.abs(a_cubic)):.2f} deg/s²")

### Questions for Exercise 2

✍️ **Answer these questions:**

1. Which profile (trapezoidal or cubic) has higher peak acceleration?
2. What are the advantages of trapezoidal profiles?
3. What happens if $T$ is too small to reach $v_{max}$?
4. In industrial robots, why are velocity/acceleration limits important?

---

## Exercise 3: Multi-Segment Trajectories (Via Points)

### Objectives
- Plan trajectories through multiple waypoints
- Ensure velocity continuity at via points
- Apply to multi-joint robots

### Problem Statement

A 2-DOF robot must move through 3 waypoints:

- Waypoint 1: $q_1 = [0°, 0°]$ at $t=0$
- Waypoint 2: $q_2 = [45°, 30°]$ at $t=2s$
- Waypoint 3: $q_3 = [90°, 60°]$ at $t=4s$

Use cubic splines to ensure smooth transitions.

**Part A: Understanding Via Points**

For $n$ waypoints, we have $n-1$ segments. Each segment is a cubic polynomial.

Constraints:
- Position at each waypoint
- Velocity continuity at interior waypoints
- Zero velocity at start and end (optional)

**Part B: Implementation**

In [ ]:
# Define waypoints
waypoints = np.array([
    [0, 0],      # q1 at t=0
    [45, 30],    # q2 at t=2
    [90, 60]     # q3 at t=4
])

time_points = np.array([0, 2, 4])  # seconds

# Use scipy CubicSpline for smooth interpolation
from scipy.interpolate import CubicSpline

# Create splines for each joint
spline_joint1 = CubicSpline(time_points, waypoints[:, 0], bc_type='clamped')  # clamped = zero vel at ends
spline_joint2 = CubicSpline(time_points, waypoints[:, 1], bc_type='clamped')

# Evaluate trajectory
t_vec = np.linspace(0, 4, 200)
q1_traj = spline_joint1(t_vec)
q2_traj = spline_joint2(t_vec)
q1_dot = spline_joint1(t_vec, 1)  # 1st derivative
q2_dot = spline_joint2(t_vec, 1)
q1_ddot = spline_joint1(t_vec, 2)  # 2nd derivative
q2_ddot = spline_joint2(t_vec, 2)

# Plot
fig, axes = plt.subplots(2, 3, figsize=(15, 8))

# Joint 1
axes[0, 0].plot(t_vec, q1_traj, 'b-', linewidth=2)
axes[0, 0].plot(time_points, waypoints[:, 0], 'ro', markersize=8, label='Waypoints')
axes[0, 0].set_ylabel('Joint 1 Position (deg)')
axes[0, 0].set_title('Joint 1 Trajectory')
axes[0, 0].grid(True)
axes[0, 0].legend()

axes[0, 1].plot(t_vec, q1_dot, 'r-', linewidth=2)
axes[0, 1].set_ylabel('Joint 1 Velocity (deg/s)')
axes[0, 1].set_title('Joint 1 Velocity')
axes[0, 1].grid(True)

axes[0, 2].plot(t_vec, q1_ddot, 'g-', linewidth=2)
axes[0, 2].set_ylabel('Joint 1 Accel (deg/s²)')
axes[0, 2].set_title('Joint 1 Acceleration')
axes[0, 2].grid(True)

# Joint 2
axes[1, 0].plot(t_vec, q2_traj, 'b-', linewidth=2)
axes[1, 0].plot(time_points, waypoints[:, 1], 'ro', markersize=8, label='Waypoints')
axes[1, 0].set_xlabel('Time (s)')
axes[1, 0].set_ylabel('Joint 2 Position (deg)')
axes[1, 0].set_title('Joint 2 Trajectory')
axes[1, 0].grid(True)
axes[1, 0].legend()

axes[1, 1].plot(t_vec, q2_dot, 'r-', linewidth=2)
axes[1, 1].set_xlabel('Time (s)')
axes[1, 1].set_ylabel('Joint 2 Velocity (deg/s)')
axes[1, 1].set_title('Joint 2 Velocity')
axes[1, 1].grid(True)

axes[1, 2].plot(t_vec, q2_ddot, 'g-', linewidth=2)
axes[1, 2].set_xlabel('Time (s)')
axes[1, 2].set_ylabel('Joint 2 Accel (deg/s²)')
axes[1, 2].set_title('Joint 2 Acceleration')
axes[1, 2].grid(True)

plt.tight_layout()
plt.show()

**Part C: Verification with Robotics Toolbox**

Use `jtraj()` function from robotics-toolbox.

In [ ]:
# Using robotics toolbox jtraj (joint trajectory)
q0 = np.deg2rad(waypoints[0, :])  # Start position
q1 = np.deg2rad(waypoints[1, :])  # First via point
q2 = np.deg2rad(waypoints[2, :])  # End position

# Generate trajectory from q0 to q1
traj1 = rtb.jtraj(q0, q1, 50)  # 50 time steps

# Generate trajectory from q1 to q2
traj2 = rtb.jtraj(q1, q2, 50)

# Combine trajectories
q_rtb = np.vstack([traj1.q, traj2.q])
qd_rtb = np.vstack([traj1.qd, traj2.qd])

# Plot
t_rtb = np.linspace(0, 4, len(q_rtb))

plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(t_rtb, np.rad2deg(q_rtb[:, 0]), 'b-', linewidth=2, label='Joint 1')
plt.plot(t_rtb, np.rad2deg(q_rtb[:, 1]), 'r-', linewidth=2, label='Joint 2')
plt.xlabel('Time (s)')
plt.ylabel('Position (deg)')
plt.title('Joint Trajectories (Robotics Toolbox)')
plt.legend()
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(t_rtb, np.rad2deg(qd_rtb[:, 0]), 'b-', linewidth=2, label='Joint 1')
plt.plot(t_rtb, np.rad2deg(qd_rtb[:, 1]), 'r-', linewidth=2, label='Joint 2')
plt.xlabel('Time (s)')
plt.ylabel('Velocity (deg/s)')
plt.title('Joint Velocities (Robotics Toolbox)')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

print("Note: Velocity discontinuity at via point is visible!")
print("For smoother motion, use mstraj() or mtraj()")

### Questions for Exercise 3

✍️ **Answer these questions:**

1. Why is velocity continuity important at via points?
2. What happens if we use linear interpolation between waypoints?
3. How would you add a "pause" at a waypoint?
4. Compare `jtraj()` vs `CubicSpline` - which is smoother?

---

# END OF SESSION 1

**Before Session 2:**
- Review your analytical solutions
- Make sure all plots are clear and labeled
- Answer the questions for each exercise
- Think about how trajectories relate to real robot motion

---